In [26]:
import sys
assert sys.version_info >= (3, 5)
# Python ≥3.5 is required
import sklearn
assert sklearn.__version__ >= "0.20"
import numpy as np
import os
import tarfile
import urllib
import pandas as pd
import urllib.request
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn import preprocessing
from sklearn import metrics
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
from pathlib import Path

In [27]:
data = pd.read_csv("02-19-2023.csv")

In [28]:
data

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2023-02-20 04:21:05,33.939110,67.709953,209107,7896,NaN,NaN,Afghanistan,537.158776,3.776057
1,NaN,NaN,NaN,Albania,2023-02-20 04:21:05,41.153300,20.168300,334314,3596,NaN,NaN,Albania,11616.999097,1.075635
2,NaN,NaN,NaN,Algeria,2023-02-20 04:21:05,28.033900,1.659600,271425,6881,NaN,NaN,Algeria,618.970454,2.535139
3,NaN,NaN,NaN,Andorra,2023-02-20 04:21:05,42.506300,1.521800,47866,165,NaN,NaN,Andorra,61950.430337,0.344712
4,NaN,NaN,NaN,Angola,2023-02-20 04:21:05,-11.202700,17.873900,105184,1931,NaN,NaN,Angola,320.036336,1.835831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4011,NaN,NaN,NaN,West Bank and Gaza,2023-02-20 04:21:05,31.952200,35.233200,703228,5708,NaN,NaN,West Bank and Gaza,13784.956961,0.811686
4012,NaN,NaN,NaN,Winter Olympics 2022,2023-02-20 04:21:05,39.904200,116.407400,535,0,NaN,NaN,Winter Olympics 2022,NaN,0.000000
4013,NaN,NaN,NaN,Yemen,2023-02-20 04:21:05,15.552727,48.516388,11945,2159,NaN,NaN,Yemen,40.048994,18.074508
4014,NaN,NaN,NaN,Zambia,2023-02-20 04:21:05,-13.133897,27.849332,342317,4051,NaN,NaN,Zambia,1862.042098,1.183406


In [29]:
geoJSONDict = {

  "type": "FeatureCollection",
  "crs": {
    "type": "name",
    "properties": {
      "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
    }
  },
  "features": []
}


In [69]:
# incident rate = (Cases/population)*100,000
# population = (Cases*100,000)/incident rate
data["Population"] = (data["Confirmed"] * 100000)/data["Incident_Rate"]
data["Cases_per_1000"] = (data["Confirmed"]/data['Population'])*1000
#data["Incident_Rate"] = (data["Deaths"] / data["Population"])* 1000
data["Case_Fatality_Ratio"] = (data["Case_Fatality_Ratio"]*100).round()
data = data[data['Confirmed'] >= data['Deaths']]
data = data.sort_values(by='Incident_Rate', ascending=False)
data.head(50)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population,Cases_per_1000
0,NaN,NaN,Faroe Islands,Denmark,2023-02-20 04:21:05,61.892600,-6.911800,34658.0,28.0,NaN,NaN,"Faroe Islands, Denmark",70926.020669,800.0,48865.0,709.260207
1,NaN,NaN,NaN,San Marino,2023-02-20 04:21:05,43.942400,12.457800,23521.0,122.0,NaN,NaN,San Marino,69305.792916,5200.0,33938.0,693.057929
2,NaN,NaN,NaN,Austria,2023-02-20 04:21:05,47.516200,14.550100,5863481.0,21820.0,NaN,NaN,Austria,65103.493072,3700.0,9006400.0,651.034931
3,NaN,NaN,NaN,Slovenia,2023-02-20 04:21:05,46.151200,14.995500,1325505.0,7067.0,NaN,NaN,Slovenia,63758.939686,5300.0,2078932.0,637.589397
4,NaN,NaN,NaN,Brunei,2023-02-20 04:21:05,4.535300,114.727700,277441.0,225.0,NaN,NaN,Brunei,63417.549939,800.0,437483.0,634.175499
5,NaN,NaN,NaN,Andorra,2023-02-20 04:21:05,42.506300,1.521800,47866.0,165.0,NaN,NaN,Andorra,61950.430337,3400.0,77265.0,619.504303
6,NaN,NaN,NaN,Iceland,2023-02-20 04:21:05,64.963100,-19.020800,208921.0,211.0,NaN,NaN,Iceland,61222.271062,1000.0,341250.0,612.222711
7,NaN,NaN,Martinique,France,2023-02-20 04:21:05,14.641500,-61.024200,228756.0,1091.0,NaN,NaN,"Martinique, France",60958.522644,4800.0,375265.0,609.585226
8,NaN,NaN,Jersey,United Kingdom,2023-02-20 04:21:05,49.213800,-2.135800,66391.0,161.0,NaN,NaN,"Jersey, United Kingdom",60741.994511,2400.0,109300.0,607.419945
9,NaN,NaN,Gibraltar,United Kingdom,2023-02-20 04:21:05,36.140800,-5.353600,20414.0,111.0,NaN,NaN,"Gibraltar, United Kingdom",60591.849455,5400.0,33691.0,605.918495


In [70]:
data.loc[data['Country_Region']=='United Arab Emirates']

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population,Cases_per_1000
449,NaN,NaN,NaN,United Arab Emirates,2023-02-20 04:21:05,23.424076,53.847818,1051079.0,2349.0,NaN,NaN,United Arab Emirates,10627.264822,2200.0,9890400.0,106.272648


In [32]:
data["Deaths"].sum()

6854432

In [33]:
data["Case_Fatality_Ratio"].sum()

574322.0

In [34]:
data["Incident_Rate"].sum()

111222738.00460437

In [35]:
us_data = data.loc[data['Country_Region']=='US']
data = data[data['Country_Region']!='US']

In [36]:
us_data

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Population
3495,48301.0,Loving,Texas,US,2023-02-20 04:21:05,31.849476,-103.581857,396,1,NaN,NaN,"Loving, Texas, US",234319.526627,25.0,169.0
755,2068.0,Denali,Alaska,US,2023-02-20 04:21:05,63.672640,-150.007611,1801,2,NaN,NaN,"Denali, Alaska, US",85884.597043,11.0,2097.0
765,2180.0,Nome,Alaska,US,2023-02-20 04:21:05,64.903207,-164.035380,7547,7,NaN,NaN,"Nome, Alaska, US",75439.824070,9.0,10004.0
767,2188.0,Northwest Arctic,Alaska,US,2023-02-20 04:21:05,67.049192,-159.750395,5747,22,NaN,NaN,"Northwest Arctic, Alaska, US",75410.051174,38.0,7621.0
3468,48247.0,Jim Hogg,Texas,US,2023-02-20 04:21:05,27.044539,-98.696819,3792,23,NaN,NaN,"Jim Hogg, Texas, US",72923.076923,61.0,5200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3753,90051.0,Unassigned,Virginia,US,2020-12-21 13:27:30,NaN,NaN,0,0,NaN,NaN,"Unassigned, Virginia, US",NaN,NaN,NaN
3798,90053.0,Unassigned,Washington,US,2023-02-20 04:21:05,NaN,NaN,3303,6,NaN,NaN,"Unassigned, Washington, US",NaN,18.0,NaN
3852,90054.0,Unassigned,West Virginia,US,2021-07-31 23:21:38,NaN,NaN,0,0,NaN,NaN,"Unassigned, West Virginia, US",NaN,NaN,NaN
3922,90055.0,Unassigned,Wisconsin,US,2023-02-20 04:21:05,NaN,NaN,0,0,NaN,NaN,"Unassigned, Wisconsin, US",NaN,NaN,NaN


In [37]:
country_wise_data = pd.DataFrame()

In [38]:
#data struct to store states
# for every state in us_data: 
# get average of Case_Fatality_Ratio, median of lat and long, country
# insert row into data

countries =  data.Country_Region.values.tolist()
countries = list(dict.fromkeys(countries))

for country in countries:
    tempDf = data.loc[data['Country_Region']==country]
    countryDf = tempDf.groupby('Country_Region', as_index=False).Deaths.sum()
    case_fatality = tempDf.groupby('Country_Region', as_index=False).Case_Fatality_Ratio.median().iloc[0]['Case_Fatality_Ratio']
    lat = tempDf.groupby('Country_Region', as_index=False).Lat.median().iloc[0]['Lat']
    long = tempDf.groupby('Country_Region', as_index=False).Long_.median().iloc[0]['Long_']
  #  country = "US"
    countryDf["Lat"] = lat
    countryDf["Long_"] = long
    countryDf["Case_Fatality_Ratio"] = case_fatality
   # stateDf["Country_Region"] = country
    countryDf
    country_wise_data = country_wise_data.append(countryDf, ignore_index=True)

C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  country_wise_data = country_wise_data.append(countryDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1714300058.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

In [39]:
country_wise_data_scaled = country_wise_data
country_wise_data.sort_values(by='Case_Fatality_Ratio', ascending=False).head(100)

,Country_Region,Deaths,Lat,Long_,Case_Fatality_Ratio
196,MS Zaandam,2,NaN,NaN,2222.0
192,Yemen,2159,15.552727,48.516388,1807.0
185,Sudan,5011,12.862800,30.217600,786.0
171,Syria,3164,34.802075,38.996815,551.0
181,Somalia,1361,5.152149,46.199616,498.0
...,...,...,...,...,...
70,India,530761,23.780432,78.942140,107.0
139,Venezuela,5852,6.423800,-66.589700,106.0
147,Equatorial Guinea,183,1.650800,10.267900,106.0
35,Chile,64085,-35.046900,-71.475800,104.0


In [40]:
country_wise_data.quantile(0.25)

Deaths                 380.0000
Lat                      3.9193
Long_                   -8.2245
Case_Fatality_Ratio     47.5000
Name: 0.25, dtype: float64

In [41]:
country_wise_data.median()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\2100298474.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  country_wise_data.median()


Deaths                 3164.0000
Lat                      17.0608
Long_                    20.9394
Case_Fatality_Ratio     103.0000
dtype: float64

In [42]:
country_wise_data.quantile(0.75).round()

Deaths                 18506.0
Lat                       39.0
Long_                     47.0
Case_Fatality_Ratio      187.0
Name: 0.75, dtype: float64

In [43]:
country_wise_data_scaled["Case_Fatality_Ratio"] = country_wise_data_scaled["Case_Fatality_Ratio"]/2.28
country_wise_data_scaled["Deaths"] = country_wise_data_scaled["Deaths"]/2.28
country_wise_data.sort_values(by='Deaths', ascending=False).tail(100)

,Country_Region,Deaths,Lat,Long_,Case_Fatality_Ratio
171,Syria,1387.719298,34.802075,38.996815,241.666667
186,Nigeria,1383.771930,9.082000,8.675300,51.754386
151,Cambodia,1340.350877,11.550000,104.916700,96.491228
131,Kyrgyzstan,1311.842105,41.204380,74.766098,63.596491
27,Estonia,1283.333333,58.595300,25.013600,21.052632
...,...,...,...,...,...
60,Tuvalu,0.000000,-7.109500,177.649300,0.000000
194,Antarctica,0.000000,-71.949900,23.347000,0.000000
128,Holy See,0.000000,41.902900,12.453400,0.000000
197,Summer Olympics 2020,0.000000,35.649100,139.773700,0.000000


In [44]:
country_wise_data_scaled.quantile(0.25)

Deaths                 166.666667
Lat                      3.919300
Long_                   -8.224500
Case_Fatality_Ratio     20.833333
Name: 0.25, dtype: float64

In [45]:
country_wise_data_scaled.median()

C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\2592085398.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  country_wise_data_scaled.median()


Deaths                 1387.719298
Lat                      17.060800
Long_                    20.939400
Case_Fatality_Ratio      45.175439
dtype: float64

In [46]:
country_wise_data_scaled.quantile(0.75).round()

Deaths                 8117.0
Lat                      39.0
Long_                    47.0
Case_Fatality_Ratio      82.0
Name: 0.75, dtype: float64

In [47]:
#data struct to store states
# for every state in us_data: 
# get average of Case_Fatality_Ratio, median of lat and long, country
# insert row into data

states =  us_data.Province_State.values.tolist()
states = list(dict.fromkeys(states))

for state in states:
    tempDf = us_data.loc[us_data['Province_State']==state]
    stateDf = tempDf.groupby('Province_State', as_index=False).Case_Fatality_Ratio.mean()
    lat = tempDf.groupby('Province_State', as_index=False).Lat.median().iloc[0]['Lat']
    long = tempDf.groupby('Province_State', as_index=False).Long_.median().iloc[0]['Long_']
  #  country = "US"
    stateDf["Lat"] = lat
    stateDf["Long_"] = long
   # stateDf["Country_Region"] = country
    stateDf
    data = data.append(stateDf, ignore_index=True)

C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\U

C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\Users\Shawn\AppData\Local\Temp\ipykernel_5228\1471081376.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(stateDf, ignore_index=True)
C:\U

In [48]:
clean_data_case_fatality = data.drop(['FIPS', 'Admin2','Province_State','Country_Region','Last_Update','Incident_Rate','Recovered','Active','Combined_Key','Deaths','Confirmed'], axis=1)
clean_data_case_fatality = clean_data_case_fatality.dropna()
clean_data_case_fatality.loc[~(clean_data_case_fatality==0.0).all(axis=1)]
clean_data_case_fatality = clean_data_case_fatality.sort_values(by='Case_Fatality_Ratio', ascending=False)
clean_data_case_fatality.head(50)

,Lat,Long_,Case_Fatality_Ratio,Population
688,15.552727,48.516388,1807.0,29825968.0
605,19.566500,-101.706800,789.0,4825401.0
675,12.862800,30.217600,786.0,43849269.0
588,19.041400,-98.206300,764.0,6604451.0
599,19.173800,-96.134200,731.0,8539862.0
464,-6.353049,-79.824113,730.0,1310800.0
485,-5.133361,-80.335861,729.0,2048000.0
505,-9.621718,-73.444929,720.0,589100.0
558,30.840600,-115.283800,719.0,3634868.0
392,-14.235097,-75.574821,700.0,975200.0


In [49]:
clean_data_deaths = data.drop(['FIPS', 'Admin2','Province_State','Country_Region','Last_Update','Incident_Rate','Recovered','Active','Combined_Key','Case_Fatality_Ratio','Confirmed'], axis=1)
clean_data_deaths = clean_data_deaths.dropna()
clean_data_deaths.loc[~(clean_data_deaths==0.0).all(axis=1)]
clean_data_deaths = clean_data_deaths.sort_values(by='Deaths', ascending=False)
clean_data_deaths.head(50)

,Lat,Long_,Deaths,Population
118,52.355500,-1.174300,184819.0,55977200.0
373,-23.550500,-46.633300,178840.0,45919049.0
12,46.227600,2.213700,161090.0,65273512.0
603,-0.789300,113.921300,160886.0,273523621.0
525,19.449759,76.108221,148421.0,123144223.0
479,32.427908,53.688046,144804.0,83992953.0
262,-38.416100,-63.616700,130448.0,45195777.0
321,51.919400,19.145100,118826.0,37846605.0
521,-30.559500,22.937500,102595.0,59308690.0
284,38.963700,35.243300,101492.0,84339067.0


In [50]:
#The mistake made here is Im rendering EACH AND EVERY covid case 
#Instead of that, create an object with long, lat and total case count.
#Hence we should only have approx 4000 json objects instead of 600 million+
#TODO: Figure out how mapbox is gonna render it



for i in range(len(clean_data_case_fatality)):
    
    longitude = clean_data_case_fatality.iloc[i]['Long_']
    latitude = clean_data_case_fatality.iloc[i]['Lat']
    data_points = int(clean_data_case_fatality.iloc[i]['Case_Fatality_Ratio'])
    
    
    for x in range(data_points):
        geometry = {
        "geometry": {
          "coordinates": [
            longitude,
            latitude
          ]
        }
      }
        geoJSONDict["features"].append(geometry)
    

json_object = json.dumps(geoJSONDict, separators=(',', ":"))
with open("../mapbox-protype2/src/data/case_fatality_ratio.json", "w") as outfile:
    outfile.write(json_object)

    
geoJSONDict = {

  "type": "FeatureCollection",
  "crs": {
    "type": "name",
    "properties": {
      "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
    }
  },
  "features": []
}
    
    
    

for i in range(len(clean_data_deaths)):
    
    longitude = clean_data_deaths.iloc[i]['Long_']
    latitude = clean_data_deaths.iloc[i]['Lat']
    data_points = int(clean_data_deaths.iloc[i]['Deaths'])
    
    
    for x in range(data_points):
        geometry = {
        "geometry": {
          "coordinates": [
            longitude,
            latitude
          ]
        }
      }
        geoJSONDict["features"].append(geometry)
    

json_object = json.dumps(geoJSONDict, separators=(',', ":"))
with open("../mapbox-protype2/src/data/deaths.json", "w") as outfile:
    outfile.write(json_object)
    

## Creating the graphs

In [51]:
vaccination_data = pd.read_csv("vaccinations-by-manufacturer.csv")

In [52]:
vaccination_data

,location,date,vaccine,total_vaccinations
0,Argentina,2020-12-29,Oxford/AstraZeneca,1
1,Argentina,2020-12-29,Sinopharm/Beijing,1
2,Argentina,2020-12-29,Sputnik V,20491
3,Argentina,2020-12-30,Sputnik V,40592
4,Argentina,2020-12-31,Sputnik V,43398
...,...,...,...,...
57665,European Union,2023-03-21,Sanofi/GSK,4532
57666,European Union,2023-03-21,Sinopharm/Beijing,2319921
57667,European Union,2023-03-21,Sinovac,8891
57668,European Union,2023-03-21,Sputnik V,1845386


In [53]:
vaccination_data.dropna()

,location,date,vaccine,total_vaccinations
0,Argentina,2020-12-29,Oxford/AstraZeneca,1
1,Argentina,2020-12-29,Sinopharm/Beijing,1
2,Argentina,2020-12-29,Sputnik V,20491
3,Argentina,2020-12-30,Sputnik V,40592
4,Argentina,2020-12-31,Sputnik V,43398
...,...,...,...,...
57665,European Union,2023-03-21,Sanofi/GSK,4532
57666,European Union,2023-03-21,Sinopharm/Beijing,2319921
57667,European Union,2023-03-21,Sinovac,8891
57668,European Union,2023-03-21,Sputnik V,1845386


In [54]:
vaccination_data.drop(['location','date'],axis=1)

,vaccine,total_vaccinations
0,Oxford/AstraZeneca,1
1,Sinopharm/Beijing,1
2,Sputnik V,20491
3,Sputnik V,40592
4,Sputnik V,43398
...,...,...
57665,Sanofi/GSK,4532
57666,Sinopharm/Beijing,2319921
57667,Sinovac,8891
57668,Sputnik V,1845386


In [55]:
vaccines_list = vaccination_data.vaccine.unique().tolist()
vaccine_dict = {}
for i in range(len(vaccines_list)):
    vaccine_dict[vaccines_list[i]] = 0
vaccine_dict

{'Oxford/AstraZeneca': 0,
 'Sinopharm/Beijing': 0,
 'Sputnik V': 0,
 'Pfizer/BioNTech': 0,
 'Moderna': 0,
 'CanSino': 0,
 'Sputnik Light': 0,
 'Johnson&Johnson': 0,
 'Novavax': 0,
 'Sanofi/GSK': 0,
 'Valneva': 0,
 'Medicago': 0,
 'Sinovac': 0,
 'Covaxin': 0,
 'SKYCovione': 0}

In [56]:
vaccine_dict = {'Oxford/AstraZeneca': 0,
 'Sinopharm/Beijing': 0,
 'Pfizer/BioNTech': 0,
 'Moderna': 0,
 'Johnson&Johnson': 0,
 'Others':0}
vaccine_dict

{'Oxford/AstraZeneca': 0,
 'Sinopharm/Beijing': 0,
 'Pfizer/BioNTech': 0,
 'Moderna': 0,
 'Johnson&Johnson': 0,
 'Others': 0}

In [57]:
for i in range(len(vaccination_data)):
    currVaccine = vaccination_data.iloc[i]['vaccine']
    currCount = vaccination_data.iloc[i]['total_vaccinations']
    
    if currVaccine == 'Oxford/AstraZeneca' or currVaccine == 'Sinopharm/Beijing' or currVaccine == 'Pfizer/BioNTech' or currVaccine == 'Moderna' or currVaccine == 'Johnson&Johnson' :
        dictCount = vaccine_dict[currVaccine]
        vaccine_dict[currVaccine] = int(currCount + dictCount)
    else:
        othersCount = vaccine_dict['Others']
        vaccine_dict['Others'] = int(currCount + dictCount)

In [58]:
vaccine_dict

{'Oxford/AstraZeneca': 82093563042,
 'Sinopharm/Beijing': 31021031484,
 'Pfizer/BioNTech': 939904167926,
 'Moderna': 263357674637,
 'Johnson&Johnson': 26013600382,
 'Others': 31018721338}

In [59]:
json_object = json.dumps(vaccine_dict, separators=(',', ":"))
with open("../mapbox-protype2/src/data/vaccines.json", "w") as outfile:
    outfile.write(json_object)

In [66]:
from datetime import date

totalCases = int(data["Confirmed"].sum())
totalDeaths = int(data["Deaths"].sum())
statsDict ={
    'totalCases': f"{totalCases:,}",
    'totalDeaths':f"{totalDeaths:,}",
    'cases_per_1000':int((totalCases/data["Population"].sum())*1000),
    'lastUpdated': date.today().strftime("%d/%m/%Y")
}
statsDict

{'totalCases': '570,920,151',
 'totalDeaths': '5,745,235',
 'cases_per_1000': 77,
 'lastUpdated': '25/03/2023'}

In [67]:
json_object = json.dumps(statsDict, separators=(',', ":"))
with open("../mapbox-protype2/src/data/stats.json", "w") as outfile:
    outfile.write(json_object)

In [73]:
data['Cases_per_1000'].sum()

136448.18359682788